Find the info needed to run specfem for each event in catalogue:
* average strike
* average dip
* average rake
* fault coordinates
* mo

In [22]:
# import modules
import math

from rsqsim_api.catalogue.catalogue import RsqSimCatalogue
from rsqsim_api.fault.multifault import RsqSimMultiFault
import os
from shapely.geometry.polygon import Polygon

Read in catalogue and fault model

In [2]:
script_dir = os.path.abspath('')
fault_dir = "../../../data/shaw2021/rundir5091"
catalogue_dir = os.path.join(fault_dir,"specfem")

fault_model = RsqSimMultiFault.read_fault_file_bruce(os.path.join(script_dir, fault_dir, "zfault_Deepen.in"),
                                                     os.path.join(script_dir, fault_dir, "znames_Deepen.in"),
                                                     transform_from_utm=True)
whole_catalogue = RsqSimCatalogue.from_csv_and_arrays(os.path.join(catalogue_dir, "single_fault_200yr"))

In [3]:
whole_catalogue.assign_event_mean_sdr(fault_model)

In [14]:
#TODO: what fault geometry is needed?
#TODO: write out SDR with location, m0 and mean slip


 ...]

In [15]:
test_ev=whole_catalogue.nth_event(fault_model=fault_model,n=10)

In [28]:
test_ev.faults[0]

184.56667803574533

array([-0.99998283,  0.00586083,  0.        ])

In [10]:
test_ev.patches[0].down_dip_vector

array([ 0.,  0., -1.])

In [11]:
whole_catalogue.event_list

array([447345, 447345, 447346, ..., 448853, 448853, 448853], dtype=int32)